In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
from keras.layers import Lambda,Input,Dense,Flatten

In [6]:
from keras.models import Model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator



In [7]:
#to find all pathnames
from glob import glob

In [8]:
#resizing images according to vgg16
IMAGE_SIZE=[224,224]

In [9]:
train_path='chest_xray/train'
test_path='chest_xray/test'

In [13]:
#for downloading weights
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [14]:
#Preprocess all layers except the output layer,since that needs to be built
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)



58892288/58889256 [==============================] - 414s 7us/step


In [16]:
#We don't train the already trained layers in vgg,weights are trained with 1000 different categories
for layer in vgg.layers:
    layer.trainable=False

In [17]:
#saving no of classes in train and storing in folders
folders=glob('chest_xray/train/*')
print(folders)

['chest_xray/train\\NORMAL', 'chest_xray/train\\PNEUMONIA']


In [20]:
#flattening whatever output we have got of vgg
x=Flatten()(vgg.output)
#adding output layer with 2 categories and appending to x layers 
prediction=Dense(len(folders),activation='softmax')(x)
#print(prediction)

Tensor("dense_2/Softmax:0", shape=(?, 2), dtype=float32)


In [21]:
#creating a model
model=Model(inputs=vgg.input,output=prediction)

C:\Users\Palak agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [22]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [25]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [26]:
#creating more images in respect to the same training dataset
train_img=ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)
test_img=ImageDataGenerator(rescale=1./255)

In [27]:
train_set=train_img.flow_from_directory('chest_xray/train',target_size=(224,224),batch_size=32,class_mode='categorical')
test_set=test_img.flow_from_directory('chest_xray/test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
#fit the model
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
162/163 [============================>.] - ETA: 23s - loss: 0.2180 - accuracy: 0.9176

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf

In [ ]:
from keras.models import load_model

In [ ]:
model.save('model_vgg19.h5')